# Prepare Optimized Model for Deployment

# Freeze Fully Optimized Graph

In [ ]:
from tensorflow.python.tools import freeze_graph

model_parent_path = './models/workshop/linear/cpu'

model_graph_path = '%s/unoptimized_cpu.pb' % model_parent_path
frozen_model_graph_path = '%s/frozen_model_graph_cpu.pb' % model_parent_path
model_checkpoint_path = '%s/model.ckpt' % model_parent_path

freeze_graph.freeze_graph(input_graph=model_graph_path, 
                          input_saver="",
                          input_binary=True, 
                          input_checkpoint=model_checkpoint_path,
                          output_node_names="add",
                          restore_op_name="save/restore_all", 
                          filename_tensor_name="save/Const:0",
                          output_graph=frozen_model_graph_path, 
                          clear_devices=True, 
                          initializer_nodes="")
print(frozen_model_graph_path)

### File Size

In [ ]:
%%bash

ls -l ./models/workshop/linear/cpu/

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import re
from google.protobuf import text_format
from tensorflow.core.framework import graph_pb2

def convert_graph_to_dot(input_graph, output_dot, is_input_graph_binary):
    graph = graph_pb2.GraphDef()
    with open(input_graph, "rb") as fh:
        if is_input_graph_binary:
            graph.ParseFromString(fh.read())
        else:
            text_format.Merge(fh.read(), graph)
    with open(output_dot, "wt") as fh:
        print("digraph graphname {", file=fh)
        for node in graph.node:
            output_name = node.name
            print("  \"" + output_name + "\" [label=\"" + node.op + "\"];", file=fh)
            for input_full_name in node.input:
                parts = input_full_name.split(":")
                input_name = re.sub(r"^\^", "", parts[0])
                print("  \"" + input_name + "\" -> \"" + output_name + "\";", file=fh)
        print("}", file=fh)
        print("Created dot file '%s' for graph '%s'." % (output_dot, input_graph))
        

In [ ]:

input_graph='./models/workshop/linear/cpu/frozen_model_graph_cpu.pb'
output_dot='./frozen_model_graph_cpu.dot'
convert_graph_to_dot(input_graph=input_graph, output_dot=output_dot, is_input_graph_binary=True)

In [ ]:
%%bash

dot -T png ./frozen_model_graph_cpu.dot \
    -o ./frozen_model_graph_cpu.png > /tmp/a.out

In [ ]:
from IPython.display import Image

Image('./frozen_model_graph_cpu.png')

# Save Model for Deployment and Inference

## Reset Default Graph

In [ ]:
import tensorflow as tf

tf.reset_default_graph()

## Create New Session

In [ ]:
sess = tf.Session()

## Load Frozen Graph

In [ ]:
from tensorflow.python.tools import inspect_checkpoint

inspect_checkpoint.print_tensors_in_checkpoint_file(file_name="./models/workshop/linear/cpu/model.ckpt",
                                                    tensor_name="",
                                                    all_tensors=True,
                                                    all_tensor_names=True)

In [ ]:
saver = tf.train.import_meta_graph('./models/workshop/linear/cpu/model.ckpt.meta')
saver.restore(sess, './models/workshop/linear/cpu/model.ckpt')

model_parent_path = './models/workshop/linear/cpu'
frozen_model_graph_path = '%s/frozen_model_graph_cpu.pb' % model_parent_path
print(frozen_model_graph_path)

with tf.gfile.GFile(frozen_model_graph_path, 'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())

tf.import_graph_def(
    graph_def, 
    input_map=None, 
    return_elements=None, 
    name="", 
    op_dict=None, 
    producer_op_list=None
)

print("weights = ", sess.run("weights:0"))
print("bias = ", sess.run("bias:0"))

## Create `SignatureDef` Asset for TensorFlow Serving


In [ ]:
from tensorflow.python.saved_model import utils
from tensorflow.python.saved_model import signature_constants
from tensorflow.python.saved_model import signature_def_utils

graph = tf.get_default_graph()

x_observed = graph.get_tensor_by_name('x_observed:0')
y_pred = graph.get_tensor_by_name('add:0')

inputs_map = {'inputs': x_observed}
outputs_map = {'outputs': y_pred}

predict_signature = signature_def_utils.predict_signature_def(
                inputs = inputs_map, 
                outputs = outputs_map)
print(predict_signature)

## Save Model with Assets


In [ ]:
from tensorflow.python.saved_model import builder as saved_model_builder
from tensorflow.python.saved_model import tag_constants

from shutil import rmtree


saved_model_path = './models/workshop/saved_model/linear/cpu/0'
print(saved_model_path)

rmtree(saved_model_path)

builder = saved_model_builder.SavedModelBuilder(saved_model_path)
builder.add_meta_graph_and_variables(sess, 
                                     [tag_constants.SERVING],
                                     signature_def_map={'predict':predict_signature,                                     
signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY:predict_signature}, 
                                     clear_devices=True,
)

builder.save(as_text=False)

In [ ]:
import os
print(saved_model_path)
os.listdir(saved_model_path)
os.listdir('%s/variables' % saved_model_path)

In [ ]:
%%bash
echo "./models/workshop/saved_model/linear/cpu/0"
echo ""
ls -al ./models/workshop/saved_model/linear/cpu/0

## Inspect with [Saved Model CLI](https://www.tensorflow.org/programmers_guide/saved_model_cli)
Note:  This takes a minute or two for some reason.  Please be patient.

In [ ]:
import subprocess

output = subprocess.run(["saved_model_cli", "show", \
                "--dir", saved_model_path, "--all"], \
                stdout=subprocess.PIPE,
                stderr=subprocess.PIPE)

print(output.stdout.decode('utf-8'))

## Predict with Python (SLOW)

In [ ]:
from tensorflow.contrib import predictor
import numpy as np

input_shape = 1
input_data = np.random.random_sample(input_shape)

predict_fn = predictor.from_saved_model(saved_model_path)

In [ ]:
%%time
predictions = predict_fn({'inputs': input_data})

In [ ]:
print('Prediction: %s' % predictions["outputs"])

## (Optional) Saved Model -- Simple Save
Note:  This takes a minute or two for some reason.  Please be patient.

In [ ]:
import tensorflow.saved_model as saved_model

from shutil import rmtree

simple_saved_model_path = './models/workshop/simple_saved_model/linear/cpu/0'
print(simple_saved_model_path)

rmtree(simple_saved_model_path)

saved_model.simple_save(sess,
            simple_saved_model_path,
            inputs={'inputs': x_observed},
            outputs={"outputs": y_pred})

In [ ]:
sess.close()

In [ ]:
import subprocess

output = subprocess.run(["saved_model_cli", "show", \
                "--dir", simple_saved_model_path, "--all"], \
                stdout=subprocess.PIPE,
                stderr=subprocess.PIPE)

print(output.stdout.decode('utf-8'))

## (Optional) Optimize with TensorFlow Lite

![PipelineAI + TensorFlow Lite](https://pipeline.ai/assets/img/toco-optimizer.png)

In [ ]:
import tensorflow as tf
import os


from shutil import rmtree

tflite_model_path = './models/workshop/tflite/linear/cpu/0'
print(tflite_model_path)

rmtree(tflite_model_path)

os.makedirs(tflite_model_path, exist_ok=True)

converter = tf.contrib.lite.TocoConverter.from_saved_model(simple_saved_model_path)

tflite_model = converter.convert()

open('%s/tflite_optimized_model.tflite' % tflite_model_path, "wb").write(tflite_model)

In [ ]:
%%bash -s "$tflite_model_path"
echo "ls -al $1"
echo ""
ls -al $1

In [ ]:
import numpy as np
import tensorflow as tf

# Load TFLite model and allocate tensors.
interpreter = tf.contrib.lite.Interpreter(model_path='%s/tflite_optimized_model.tflite' % tflite_model_path)
interpreter.allocate_tensors()

In [ ]:
# Get input and output tensors.
input_details = interpreter.get_input_details()
print('Input Tensor Details: %s' % input_details)

output_details = interpreter.get_output_details()
print('Output Tensor Details: %s' % output_details)

In [ ]:
# Test model on random input data.
input_shape = input_details[0]['shape']
input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)
print('Input: %s' % input_data)
interpreter.set_tensor(input_details[0]['index'], input_data)

In [ ]:
%%time
interpreter.invoke()

In [ ]:
output_data = interpreter.get_tensor(output_details[0]['index'])
print('Prediction: %s' % output_data)